In [71]:
datapath = "/data/jyk/aac_dataset/clotho/validation/01 A pug struggles to breathe 1_14_2008.wav"

In [72]:
from datasets import Audio

In [73]:
from encodec import EncodecModel
from encodec.utils import convert_audio

import torchaudio
import torch

In [74]:
model = EncodecModel.encodec_model_24khz()


In [75]:
from utils import count_parameters
print(count_parameters(model))

14851810


In [76]:
model.set_target_bandwidth(6.0)

In [77]:
wav, sr = torchaudio.load(datapath)
print(wav.shape[-1]/sr)
wav = convert_audio(wav, sr, model.sample_rate, model.channels)
wav = wav.unsqueeze(0)
print(sr)
print(model.sample_rate)
print(model.channels)
print(wav.shape[-1]/model.sample_rate)

18.8
44100
24000
1
18.8


In [78]:
with torch.no_grad():
    encoded_frames = model.encode(wav)

In [79]:
print(encoded_frames)

[(tensor([[[675, 798, 635,  ..., 281, 169, 457],
         [184, 740, 961,  ..., 603, 831, 857],
         [996, 832, 967,  ..., 273, 599, 771],
         ...,
         [763, 611, 140,  ...,  18,  95, 918],
         [938, 862, 674,  ..., 661, 193, 364],
         [412, 326, 339,  ..., 614, 424, 428]]]), None)]


In [80]:
print(encoded_frames[0][0].shape)

torch.Size([1, 8, 1410])


In [81]:
codes = torch.cat([encoded[0] for encoded in encoded_frames], dim=-1)  # [B, n_q, T]

In [82]:
print(codes.shape)

torch.Size([1, 8, 1410])


In [83]:
print(codes[:,1, :])

tensor([[184, 740, 961,  ..., 603, 831, 857]])


In [84]:
print(codes.transpose(1,2)[:,:,1].min())

tensor(0)


In [85]:
code_1 = codes+1

In [86]:
print(code_1[:,1, :])

tensor([[185, 741, 962,  ..., 604, 832, 858]])


In [88]:
decoded_wav = model.decode(encoded_frames)

In [89]:
decoded_wav.shape

torch.Size([1, 1, 451200])

In [92]:
decoded_wav = decoded_wav.squeeze().squeeze().detach().numpy()

In [90]:
from IPython.display import Audio

In [93]:
Audio(decoded_wav, rate=24000)